![Factories creating emissions](pollution.jpg)

Photo by Maxim Tolchinskiy on Unsplash
  
When factoring heat generation required for the manufacturing and transportation of products, _Greenhouse gas emissions attributable to products, from food to sneakers to appliances, make up more than 75% of global emissions._ (`Source: The Carbon Catalogue https://www.nature.com/articles/s41597-022-01178-9`)

Our data, which is publicly available on nature.com, contains product carbon footprints (PCFs) for various companies. PCFs are the greenhouse gas emissions attributable to a given product, measured in CO<sub>2</sub> (carbon dioxide equivalent).
<!--https://www.nature.com/articles/s41597-022-01178-9-->

This data is stored in a PostgreSQL database containing one table, `product_emissions`, which looks at PCFs by product as well as the stage of production that these emissions occurred. Here's a snapshot of what `product_emissions` contains in each column:

### `product_emissions`

| field                              | data type |
|------------------------------------|-----------|
| `id`                                 | `VARCHAR`   |
| `year`                               | `INT`       |
| `product_name`                       | `VARCHAR`   |
| `company`                            | `VARCHAR`   |
| `country`                            | `VARCHAR`   |
| `industry_group`                     | `VARCHAR`   |
| `weight_kg`                          | `NUMERIC`   |
| `carbon_footprint_pcf`               | `NUMERIC`   |
| `upstream_percent_total_pcf`         | `VARCHAR`   |
| `operations_percent_total_pcf`       | `VARCHAR`   |
| `downstream_percent_total_pcf`       | `VARCHAR`   |

You'll use this data to examine the carbon footprint of each industry in the dataset! 

## Setting up the query from creating CTE, Filtering, and joining all queries

In [22]:
-- Setting up max_year
WITH max_year AS (
	SELECT MAX(year) AS year
	FROM product_emissions
)
-- Show the output
SELECT * FROM max_year;

,year
0,2017


In [23]:
-- Set up the columns needed for the query table
SELECT 
	industry_group, -- Lists of industries
	COUNT(DISTINCT company) AS num_companies, -- Count the unique numbers of company 
	ROUND(SUM(carbon_footprint_pcf), 1) AS total_industry_footprint -- Get the total carbon footprints per industries
FROM product_emissions
GROUP BY industry_group
LIMIT 5;

,industry_group,num_companies,total_industry_footprint
0,Automobiles & Components,8,2582263.4
1,Capital Goods,15,258632.7
2,Chemicals,8,44939.7
3,Commercial & Professional Services,4,4920.6
4,Consumer Durables & Apparel,10,7096.7


In [24]:
-- Setting up query_table
WITH query_table AS (
	SELECT 
		industry_group,
		COUNT(DISTINCT company) AS num_companies,  
		ROUND(SUM(carbon_footprint_pcf), 1) AS total_industry_footprint 
	FROM product_emissions
	GROUP BY industry_group
)

-- Show the output
SELECT * FROM query_table;


,industry_group,num_companies,total_industry_footprint
0,Automobiles & Components,8,2582263.4
1,Capital Goods,15,258632.7
2,Chemicals,8,44939.7
3,Commercial & Professional Services,4,4920.6
4,Consumer Durables & Apparel,10,7096.7
5,"Consumer Durables, Household and Personal Prod...",2,931.0
6,Containers & Packaging,4,2988.1
7,Electrical Equipment and Machinery,7,9801557.6
8,Energy,4,10774.1
9,Food & Beverage Processing,6,139.2


In [25]:
-- Getting the Final Query
WITH max_year AS (
	SELECT MAX(year) AS year
	FROM product_emissions
),
query_table AS (
	SELECT 
		industry_group,
		COUNT(DISTINCT company) AS num_companies,
		ROUND(SUM(carbon_footprint_pcf), 1) AS total_industry_footprint 
	FROM product_emissions
	WHERE year IN (SELECT year FROM max_year)
	GROUP BY industry_group
)
-- Show final query output
SELECT *
FROM query_table
ORDER BY total_industry_footprint DESC;

,industry_group,num_companies,total_industry_footprint
0,Materials,3,107129.0
1,Capital Goods,2,94942.7
2,Technology Hardware & Equipment,4,21865.1
3,"Food, Beverage & Tobacco",1,3161.5
4,Commercial & Professional Services,1,740.6
5,Software & Services,1,690.0


## Carbon Emissions by Industry

In [26]:
WITH max_year AS (
	SELECT MAX(year) AS year
	FROM product_emissions
),
query_table AS (
	SELECT 
		industry_group,
		COUNT(DISTINCT company) AS num_companies,
		ROUND(SUM(carbon_footprint_pcf), 1) AS total_industry_footprint 
	FROM product_emissions
	WHERE year IN (SELECT year FROM max_year)
	GROUP BY industry_group
)
-- Show final query output
SELECT *
FROM query_table
ORDER BY total_industry_footprint DESC;

,industry_group,num_companies,total_industry_footprint
0,Materials,3,107129.0
1,Capital Goods,2,94942.7
2,Technology Hardware & Equipment,4,21865.1
3,"Food, Beverage & Tobacco",1,3161.5
4,Commercial & Professional Services,1,740.6
5,Software & Services,1,690.0
